In [1]:
import uuid
import json
from typing import TypedDict, List, Dict, Any, Optional
from functools import partial # To pass LLM to the node
# Note: Command and interrupt from langgraph.types are not available in current version
from langgraph.types import Command, interrupt

import httpx
import requests

import mlflow
from mlflow.langchain.chat_agent_langgraph import ChatAgentState, ChatAgentToolNode
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import (
    ChatAgentChunk,
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)

from pyspark.sql.functions import col, lower, collect_set, size

# Langchain imports
from langchain_openai import AzureChatOpenAI
from langchain_community.chat_models import ChatDatabricks

# LangGraph imports
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.state import CompiledStateGraph
from langchain.schema import HumanMessage, AIMessage

# Databricks-specific imports
from databricks_langchain.genie import GenieAgent
from unitycatalog.ai.langchain.toolkit import UCFunctionToolkit
from unitycatalog.ai.core.databricks import DatabricksFunctionClient
client = DatabricksFunctionClient()

# os.environ["MLFLOW_TRACKING_URI"] = "databricks" 
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Users/malay.panigrahi@databricks.com/ML/tech_summit25/agent")

mlflow.langchain.autolog()


from databricks_langchain import ChatDatabricks

llm = ChatDatabricks(
    endpoint="databricks-claude-3-7-sonnet"
)


# Updated state to include combined_prompt and worker_outputs
class AgentState(TypedDict):
    original_prompt: str
    user_confirmed_hierarchy: Optional[str] # Material Hierarchy confirmed by the user
    extracted_material: Optional[str]
    extracted_location: Optional[str]
    user_confirmed_location_hierarchy: Optional[str] # Location Hierarchy confirmed by the user
    combined_prompt: Optional[str] # Prompt for Genie (original + confirmed hierarchy)
    final_response: Optional[str] # Kept for compatibility, but worker_outputs used now
    messages: List[Dict[str, Any]] # History of interactions
    # Dictionary to store outputs from different workers (like Genie)
    worker_outputs: Optional[Dict[str, str]]



from databricks_langchain.genie import GenieAgent

# add your genie space id here
p2p_spend_genie_space_id =  "01f0421695fe1fb694762b30f68f799d"
#capex_spend_genie_space_id =  "01f042195bd1169e8f1cb3e85a85f7e7"

p2p_spend_genie = GenieAgent(genie_space_id=p2p_spend_genie_space_id, genie_agent_name="p2p_spend_genie")

#capex_spend_genie = GenieAgent(genie_space_id=capex_spend_genie_space_id, "capex_spend_genie") 